# Extracting and Exploring C3D Metadata with Python

**Author**: Dr. Hossein Mokhtarzadeh  
**Affiliation**: [PoseIQ](https://poseiq.com)  
**Courses**: [Udemy Profile](https://www.udemy.com/user/hossein-mokhtarzadeh/)  
**LinkedIn**: [PoseIQ on LinkedIn](https://www.linkedin.com/company/poseiq)  

---

This notebook is part of *Hands-On Python for Biomechanics* by Dr. Hossein Mokhtarzadeh.  
It demonstrates how to extract, visualize, and export detailed C3D metadata using Python.  

### Step 00 — C3D Metadata Dump
In biomechanics, C3D files are widely used for motion capture data. Metadata (like POINT, ANALOG, FORCE_PLATFORM, SUBJECT blocks) provides critical context about sampling rates, units, and channel mapping. Tools like Mokka and ezc3d offer similar dumps; here, we show how to achieve the same in Python.

This step provides:
- A raw metadata overview (similar to other SW in Biomechanics).
- Interactive exploration using Jupyter widgets.
- An exportable HTML/CSV report for further analysis.
- Potential integration into PoseIQ pipelines.

---


In [ ]:
!pip install ezc3d pandas

!wget https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
new_var = %run /content/Chapter1Input.py


In [ ]:
# ==== C3D Metadata Dump (Mokka-style) ====
import ezc3d, numpy as np, pandas as pd
from pprint import pprint

# If you already have c3d loaded, reuse it; otherwise set c3d_path:
# c3d_path = "/content/Eb015pi (1).c3d"
try:
    c3d
except NameError:
    c3d = ezc3d.c3d(c3d_path)

P = c3d["parameters"]  # shorthand

def gp(key):   return P.get(key, {})
def val(g,k, default=None):
    try:
        return gp(g)[k]["value"]
    except Exception:
        return default

# ---------- File/stream info ----------
n_frames   = int(c3d["data"]["points"].shape[2])
pt_rate    = float(val("POINT","RATE", [np.nan])[0])
an_rate    = float(val("ANALOG","RATE", [np.nan])[0])
n_points   = int(val("POINT","USED", [0])[0])
n_analog   = int(val("ANALOG","USED", [0])[0])
byte_order = "IEEE Little Endian"  # ezc3d stores little-endian by default
storage    = "Integer"             # most Qualisys/Vicon exports for this file type
# If you need exact endian/storage, you can inspect the raw file header with btk/pyc3dreader.

print("=== FILE INFO ===")
print(f"Frames: 1 → {n_frames} (count {n_frames})")
print(f"Point frequency: {pt_rate} Hz")
print(f"Analog channels: {n_analog}  at  {an_rate} Hz")
print(f"Byte order: {byte_order}")
print(f"Storage: {storage}")
print("")

# ---------- POINT ----------
print("=== POINT GROUP ===")
point = {
    "X_SCREEN": val("POINT","X_SCREEN", ["?"])[0] if val("POINT","X_SCREEN", None) is not None else None,
    "Y_SCREEN": val("POINT","Y_SCREEN", ["?"])[0] if val("POINT","Y_SCREEN", None) is not None else None,
    "UNITS":    val("POINT","UNITS", ["?"])[0],
    "USED":     int(val("POINT","USED", [0])[0]),
    "FRAMES":   int(val("POINT","FRAMES", [n_frames])[0]),
    "SCALE":    float(val("POINT","SCALE", [np.nan])[0]),
    "DATA_START": int(val("POINT","DATA_START", [np.nan])[0]),
    "RATE":     float(val("POINT","RATE", [np.nan])[0]),
}
pprint(point); print("")

# ---------- ANALOG ----------
print("=== ANALOG GROUP ===")
analog = {
    "USED":       int(val("ANALOG","USED", [0])[0]),
    "RATE":       float(val("ANALOG","RATE", [np.nan])[0]),
    "GEN_SCALE":  float(val("ANALOG","GEN_SCALE", [1.0])[0]),
    "UNITS":      val("ANALOG","UNITS", []),
}
pprint(analog)

labels = list(val("ANALOG","LABELS", []))
offsets = list(val("ANALOG","OFFSET", [])) if val("ANALOG","OFFSET", None) is not None else []
scales  = list(val("ANALOG","SCALE",  [])) if val("ANALOG","SCALE",  None) is not None else []

# Build a neat table of channel metadata
df_an = pd.DataFrame({
    "label":  labels,
    "offset": offsets[:len(labels)] if offsets else None,
    "scale":  scales[:len(labels)]  if scales  else None,
    "units":  (val("ANALOG","UNITS", [])[:len(labels)] if val("ANALOG","UNITS", None) else None),
})
print("\nAnalog channels (label/offset/scale/units):")
display(df_an if "display" in globals() else df_an.head(len(labels)))

# ---------- FORCE_PLATFORM ----------
print("\n=== FORCE_PLATFORM GROUP ===")
fp = {
    "USED":        int(val("FORCE_PLATFORM","USED", [0])[0]),
    "TYPE":        list(val("FORCE_PLATFORM","TYPE", [])),
    "CORNERS_dim": np.array(val("FORCE_PLATFORM","CORNERS", [])).shape,
    "ORIGIN_dim":  np.array(val("FORCE_PLATFORM","ORIGIN", [])).shape,
    "CHANNEL_dim": np.array(val("FORCE_PLATFORM","CHANNEL", [])).shape,
    "ZERO":        list(val("FORCE_PLATFORM","ZERO", [])) if val("FORCE_PLATFORM","ZERO", None) is not None else None,
    "TRANSLATION": np.array(val("FORCE_PLATFORM","TRANSLATION", [])).tolist() if val("FORCE_PLATFORM","TRANSLATION", None) is not None else None,
    "ROTATION":    np.array(val("FORCE_PLATFORM","ROTATION", [])).tolist() if val("FORCE_PLATFORM","ROTATION", None) is not None else None,
}
pprint(fp)

# Show first plate's mapping and geometry for reference
try:
    ch = np.array(val("FORCE_PLATFORM","CHANNEL", [])).reshape(6, fp["USED"])
    print("\nCHANNEL mapping (first plate columns are 1-based indices into ANALOG):")
    print(ch[:,0].tolist())
except Exception:
    pass
try:
    corners = np.array(val("FORCE_PLATFORM","CORNERS", [])).reshape(fp["USED"], 3, 4) # (plates, xyz, four corners)
    origin  = np.array(val("FORCE_PLATFORM","ORIGIN",  [])).reshape(fp["USED"], 3)
    print("\nFirst plate CORNERS (m):")
    print(corners[0].tolist())
    print("First plate ORIGIN (m):")
    print(origin[0].tolist())
except Exception:
    pass

# ---------- SUBJECT ----------
print("\n=== SUBJECT GROUP ===")
subject = {
    "NAME":   (val("SUBJECT","NAME",   [""])[0] if val("SUBJECT","NAME", None) else None),
    "NUMBER": (int(val("SUBJECT","NUMBER", [np.nan])[0]) if val("SUBJECT","NUMBER", None) else None),
    "PROJECT":(val("SUBJECT","PROJECT",[""])[0] if val("SUBJECT","PROJECT", None) else None),
    "WEIGHT": (float(val("SUBJECT","WEIGHT",[np.nan])[0]) if val("SUBJECT","WEIGHT", None) else None),
    "HEIGHT": (float(val("SUBJECT","HEIGHT",[np.nan])[0]) if val("SUBJECT","HEIGHT", None) else None),
    "GENDER": (val("SUBJECT","GENDER", [""])[0] if val("SUBJECT","GENDER", None) else None),
    "DATE_OF_BIRTH": (val("SUBJECT","DATE_OF_BIRTH", [""])[0] if val("SUBJECT","DATE_OF_BIRTH", None) else None),
}
pprint(subject)


In [ ]:
# Interactive C3D metadata viewer (Colab/Jupyter)
# Requires: pandas, ipywidgets (both are in Colab by default)

import json, math
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as W

def _safe(v):
    try:
        if isinstance(v, (list, tuple)):
            return v
        return [v]
    except Exception:
        return [str(v)]

def _param_table(params_group):
    rows = []
    for pname, pobj in params_group.items():
        if not isinstance(pobj, dict):
            continue
        val = pobj.get("value", None)
        shape = None
        # ezc3d stores values as nested lists; infer a readable shape
        try:
            def shape_of(x):
                if isinstance(x, (list, tuple)) and len(x)>0:
                    inner = shape_of(x[0])
                    return (len(x),) + (inner if isinstance(inner, tuple) else (() if inner is None else (1,)))
                return ()
            if val is None:
                shape = ""
            else:
                shp = shape_of(val)
                shape = "×".join(map(str, shp)) if len(shp) else "1"
        except Exception:
            shape = ""
        # short preview string
        if val is None:
            preview = ""
        else:
            try:
                flat = val
                while isinstance(flat, list) and len(flat)==1:
                    flat = flat[0]
                preview = str(flat)
                if len(preview) > 80:
                    preview = preview[:77] + "..."
            except Exception:
                preview = ""
        rows.append({
            "Parameter": pname,
            "Type": pobj.get("type", ""),
            "Dim": shape,
            "Preview": preview
        })
    return pd.DataFrame(rows).sort_values("Parameter").reset_index(drop=True)

def show_c3d_metadata(c3d):
    P = c3d["parameters"]  # ezc3d-style dict

    # Left: group list + search. Right: parameter table + value viewer.
    groups = sorted([g for g in P.keys() if isinstance(P[g], dict)])
    search = W.Text(placeholder="Search parameter...", layout=W.Layout(width="100%"))
    group_list = W.Select(options=groups, rows=min(12, max(6, len(groups))), layout=W.Layout(width="100%"))
    export_btn = W.Button(description="Export CSV", icon="download")
    refresh_btn = W.Button(description="Refresh", icon="refresh")
    title = W.HTML("<h3 style='margin:6px 0'>C3D Metadata</h3>")

    # parameter table and value pane
    table_out = W.Output()
    value_out = W.Output()
    info_bar = W.HTML("", layout=W.Layout(margin="4px 0 8px 0"))

    def render_table():
        table_out.clear_output(wait=True)
        value_out.clear_output(wait=True)
        g = group_list.value
        if g is None:
            with table_out: display(HTML("<i>Select a group...</i>"))
            return
        df = _param_table(P[g])
        q = search.value.strip().lower()
        if q:
            df = df[df["Parameter"].str.lower().str.contains(q) | df["Preview"].str.lower().str.contains(q)]
        with table_out:
            # clickable table: show value on row click (via interactive widget)
            sel = W.Select(
                options=[f'{r.Parameter}   | Dim:{r.Dim}   | {r.Preview}' for r in df.itertuples()],
                rows=min(12, max(6, len(df))),
                layout=W.Layout(width="100%")
            )
            def on_pick(change):
                if change["name"] == "value":
                    idx = sel.options.index(change["new"]) if change["new"] in sel.options else None
                    if idx is None: return
                    pname = df.iloc[idx]["Parameter"]
                    pobj = P[g][pname]
                    val = pobj.get("value", None)
                    value_out.clear_output(wait=True)
                    with value_out:
                        display(HTML(f"<b>{g} / {pname}</b>"))
                        meta = {k:v for k,v in pobj.items() if k!="value"}
                        if meta: display(pd.DataFrame([meta]).T.rename(columns={0:"meta"}))
                        try:
                            print(json.dumps(val, indent=2, ensure_ascii=False)[:20000])
                        except Exception:
                            print(val)
            sel.observe(on_pick, names="value")
            display(sel)

        # update info bar
        info_bar.value = f"<small><b>Group:</b> {g} &nbsp; | &nbsp; <b>Params:</b> {len(df)}</small>"

    def on_group_change(_):
        render_table()

    def on_search_change(_):
        render_table()

    def on_export(_):
        g = group_list.value
        if not g:
            return
        df = _param_table(P[g])
        q = search.value.strip().lower()
        if q:
            df = df[df["Parameter"].str.lower().str.contains(q) | df["Preview"].str.lower().str.contains(q)]
        csv_path = f"/mnt/data/c3d_{g}_metadata.csv"
        df.to_csv(csv_path, index=False)
        value_out.clear_output(wait=True)
        with value_out:
            display(HTML(f"Saved: <a href='sandbox:{csv_path}' target='_blank'>download {g} CSV</a>"))

    group_list.observe(on_group_change, names="value")
    search.observe(on_search_change, names="value")
    export_btn.on_click(on_export)
    refresh_btn.on_click(lambda _: render_table())

    layout = W.HBox([
        W.VBox([title, search, group_list, W.HBox([export_btn, refresh_btn])],
               layout=W.Layout(min_width="280px", max_width="320px")),
        W.VBox([info_bar, table_out, W.HTML("<b>Value</b>"), value_out], layout=W.Layout(width="100%"))
    ])
    display(layout)
    if groups:
        group_list.value = groups[0]  # triggers first render

# Usage:
# show_c3d_metadata(c3d)


In [ ]:
def export_c3d_metadata(c3d, out_dir="/content", html_filename="c3d_metadata_report.html"):
    import os, json, datetime
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import html as html_mod

    def _shape_of(v):
        try:
            if isinstance(v, (list, tuple)) and len(v) > 0:
                inner = _shape_of(v[0])
                return (len(v),) + (inner if isinstance(inner, tuple) else ())
            return ()
        except Exception:
            return ()

    def _preview(v, lim=120):
        try:
            s = json.dumps(v, ensure_ascii=False,
                           default=lambda x: x.tolist() if hasattr(x, "tolist") else str(x))
        except Exception:
            s = str(v)
        return s if len(s) <= lim else s[:lim] + "..."

    def _param_rows(group_dict):
        rows = []
        for name, meta in group_dict.items():
            if not isinstance(meta, dict):
                continue
            val = meta.get("value", None)
            shp = "×".join(map(str, _shape_of(val))) if val is not None else ""
            rows.append({
                "Parameter": str(name),
                "Dim": str(shp if shp else "1"),
                "Type": str(meta.get("type", "")),
                "Preview": str(_preview(val)),
                "ValueJSON": json.dumps(
                    val, ensure_ascii=False, indent=2,
                    default=lambda x: x.tolist() if hasattr(x, "tolist")
                                      else (float(x) if isinstance(x, np.floating)
                                            else int(x) if isinstance(x, np.integer)
                                            else str(x))
                ) if val is not None else ""
            })
        rows.sort(key=lambda r: r["Parameter"].lower())
        return rows

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    P = c3d["parameters"]
    groups = sorted([g for g in P.keys() if isinstance(P[g], dict)])

    n_frames = int(c3d["data"]["points"].shape[2])
    getv = lambda g,p,d: P.get(g,{}).get(p,{}).get("value",[d])[0]
    pt_rate = float(getv("POINT","RATE", np.nan)) if "POINT" in P else np.nan
    an_used = int(getv("ANALOG","USED", 0))        if "ANALOG" in P else 0
    an_rate = float(getv("ANALOG","RATE", np.nan)) if "ANALOG" in P else np.nan

    css = """
    body{font:14px system-ui,Segoe UI,Arial;margin:0;display:flex}
    #nav{width:240px;padding:16px;border-right:1px solid #eee;position:sticky;top:0;height:100vh;overflow:auto}
    #main{flex:1;padding:20px 28px}
    h1{font-size:20px;margin:4px 0 8px} h2{margin:24px 0 8px}
    table{border-collapse:collapse;width:100%} th,td{border:1px solid #e6e6e6;padding:6px 8px;vertical-align:top}
    th{background:#fafafa} details{margin:6px 0} .muted{color:#666}
    .pill{display:inline-block;padding:2px 8px;border:1px solid #ddd;border-radius:10px;margin-right:6px}
    .search{width:100%;padding:8px;box-sizing:border-box;margin:8px 0 12px} .rowhide{display:none}
    """
    js = """
    function filterTable(inputId, tableClass){
      const q = document.getElementById(inputId).value.toLowerCase();
      document.querySelectorAll('.' + tableClass + ' tbody tr').forEach(tr=>{
        const t = tr.getAttribute('data-text') || '';
        tr.className = (q && !t.includes(q)) ? 'rowhide' : '';
      });
    }
    """

    parts = []
    parts.append(f"""<!doctype html>
<html><head><meta charset="utf-8"><title>C3D Metadata Report</title>
<style>{css}</style><script>{js}</script></head><body>
<div id="nav">
  <h1>C3D Metadata</h1>
  <div class="muted">Generated {html_mod.escape(datetime.datetime.now().isoformat(sep=' ', timespec='seconds'))}</div>
  <p><span class="pill">Frames {n_frames}</span>
     <span class="pill">Point {pt_rate} Hz</span>
     <span class="pill">Analog {an_used} ch at {an_rate} Hz</span></p>
  <h3>Groups</h3>
  <ol>""")
    for g in groups:
        parts.append(f'<li><a href="#g_{html_mod.escape(str(g))}">{html_mod.escape(str(g))}</a></li>')
    parts.append("</ol></div><div id='main'>")

    csv_paths = {}
    for g in groups:
        rows = _param_rows(P[g])
        df = pd.DataFrame(rows, columns=["Parameter","Dim","Type","Preview","ValueJSON"])
        csv_path = out_dir / f"c3d_{g}_metadata.csv"
        df.to_csv(csv_path, index=False)
        csv_paths[g] = str(csv_path)

        tbl_class = f"tbl_{g}"
        parts.append(f"<h2 id='g_{html_mod.escape(str(g))}'>{html_mod.escape(str(g))}</h2>")
        parts.append(f"<p class='muted'>CSV: <a href='c3d_{g}_metadata.csv' download>download</a></p>")
        parts.append(f"<input id='q_{g}' class='search' placeholder='Search {html_mod.escape(str(g))}' oninput=\"filterTable('q_{g}','{tbl_class}')\">")
        parts.append(f"<table class='{tbl_class}'><thead><tr><th>Parameter</th><th>Dim</th><th>Type</th><th>Preview</th><th>Value</th></tr></thead><tbody>")
        for r in rows:
            data_text = " ".join(map(str, [r["Parameter"], r["Dim"], r["Type"], r["Preview"]])).lower()
            val_html = f"<details><summary>open</summary><pre>{html_mod.escape(r['ValueJSON'])}</pre></details>" if r["ValueJSON"] else ""
            parts.append(
                f"<tr data-text='{html_mod.escape(data_text)}'>"
                f"<td>{html_mod.escape(str(r['Parameter']))}</td>"
                f"<td>{html_mod.escape(str(r['Dim']))}</td>"
                f"<td>{html_mod.escape(str(r['Type']))}</td>"
                f"<td>{html_mod.escape(str(r['Preview']))}</td>"
                f"<td>{val_html}</td>"
                f"</tr>"
            )
        parts.append("</tbody></table>")

    parts.append("</div></body></html>")
    html_path = out_dir / html_filename
    html_path.write_text("".join(parts), encoding="utf-8")
    return {"html": str(html_path), "csv": csv_paths}


In [ ]:
paths = export_c3d_metadata(c3d, out_dir="/content", html_filename="c3d_metadata_report.html")
print(paths["html"])
paths["csv"]

In [ ]:
# Open the printed HTML path in Colab’s file browser, or run:
from google.colab import files
files.download(paths["html"])

# Or if you predfer righ tin your colab with some restrictions
# Generate report + CSVs
paths = export_c3d_metadata(c3d, out_dir="/content")

# Show HTML inline in Colab
from IPython.display import HTML
with open(paths["html"], "r", encoding="utf-8") as f:
    display(HTML(f.read()))

